In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import random


In [2]:
import matplotlib.pyplot as plt

In [31]:
# Cargar datos (referencia relativa al script)
df = pd.read_csv("../input/apartments.csv")

# 3.a) LIMPIEZA DE DATOS
# Crear columna de área al cuadrado
df["area2"] = df["area"] ** 2

# Convertir columnas categóricas "yes"/"no" a binario (0/1)
for col in ["hasparkingspace", "hasbalcony", "hassecurity", "hasstorageroom", "haselevator"]:
    df[col] = df[col].apply(lambda x: 1 if x == "yes" else 0)

# Crear variables indicadoras para el dígito final del área
for i in range(10):
    df[f"end_{i}"] = (df["area"] % 10 == i).astype(int)


In [32]:
# 3b Estimación del modelo lineal
# Convertir columnas especificadas a categóricas
for col in ["month", "type", "rooms", "ownership", "buildingmaterial"]:
    df[col] = df[col].astype("category")

# Definir regresores (columnas 3 a 31 en el DataFrame original)
regressors = df.columns[2:31]  # Ajustar el índice si es necesario según las posiciones reales de las columnas
formula = "price ~ " + " + ".join(regressors)
results = ols(formula, data=df).fit()

# Guardar coeficientes de la regresión
results_df = results.summary2().tables[1]
results_df.to_csv("../output/Linear model estimation.csv")

print("El hecho de que el área de un departamento termine en 0 aumenta su precio en aproximadamente 26,904.7 comparado con terminar en 9 (la categoría base). Este coeficiente estimado parece significativo por el valor p muy bajo.")

# Partialling-out
# Lista de regresores sin end_0
Z = regressors.tolist()
Z.remove("end_0")

end_0_coef = results.params["end_0"]

# Modelo para price ~ Z
formula_price_Z = "price ~ " + " + ".join(Z)
price_Z_model = ols(formula_price_Z, data=df).fit()
price_Z_residual = price_Z_model.resid

# Modelo para end_0 ~ Z
formula_end_0_Z = "end_0 ~ " + " + ".join(Z)
end_0_Z_model = ols(formula_end_0_Z, data=df).fit()
end_0_Z_residual = end_0_Z_model.resid

# Regresión de los residuales
results_partialling_out = ols("y ~ x", data=pd.DataFrame({"x": end_0_Z_residual, "y": price_Z_residual})).fit()

# Guardar resultados de partialling-out
results_partialling_out_df = results_partialling_out.summary2().tables[1]
results_partialling_out_df.to_csv("../output/Partialling-out estimation.csv")

# El estimador del coeficiente es el mismo que al correr la regresión con todos los regresores, es decir, 26,904.7.


El hecho de que el área de un departamento termine en 0 aumenta su precio en aproximadamente 26,904.7 comparado con terminar en 9 (la categoría base). Este coeficiente estimado parece significativo por el valor p muy bajo.


In [33]:
# 3c Prima de precio para área que termina en dígito 0
# a) Regresión de end_0 sobre las demás covariables
formula_end_0 = "end_0 ~ " + " + ".join(Z)
lm_end_0 = ols(formula_end_0, data=df).fit()
residuos_end0 = lm_end_0.resid

# b) Regresión de price sobre las demás covariables
formula_precio = "price ~ " + " + ".join(Z)
lm_precio = ols(formula_precio, data=df).fit()
residuos_precio = lm_precio.resid

# c) Regresión de los residuos
lm_residuos = ols("y ~ x", data=pd.DataFrame({"x": residuos_end0, "y": residuos_precio})).fit()

# d) Comparación de coeficientes
coef_end_0 = results.params["end_0"]
coef_partial = lm_residuos.params["x"]
print("\nCOMPARACIÓN DE COEFICIENTES")
print(f"Coeficiente de end_0 en la regresión completa: {coef_end_0:.2f}")
print(f"Coeficiente de partialling-out: {coef_partial:.2f}")
print(f"¿Son iguales? {'Sí' if abs(coef_end_0 - coef_partial) < 1e-6 else 'No'}")


COMPARACIÓN DE COEFICIENTES
Coeficiente de end_0 en la regresión completa: 25147.12
Coeficiente de partialling-out: 25147.12
¿Son iguales? Sí
